In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook
import datetime as dt
from scipy.stats import norm
import requests
import json

In [ ]:
def yahoo_opt_clean(x, type):
    x = pd.io.json.json_normalize(x['optionChain']['result'][0]['options'][0][type])
    x = x[['ask', 'bid', 'expiration', 'strike', 'inTheMoney']]
    if type == 'calls':
        x['type'] = 'C'
    elif type == 'puts':
        x['type'] = 'P'
    else:
        raise ValueError('Unknown option type')
    return x


def get_options():
    url = 'https://query2.finance.yahoo.com/v7/finance/options/SPY'
    content = requests.get(url).text
    content = json.loads(content)
    current_price = content['optionChain']['result'][0]['quote']['regularMarketPrice']
    current_date = content['optionChain']['result'][0]['quote']['regularMarketTime']
    dates = content['optionChain']['result'][0]['expirationDates']
    options = yahoo_opt_clean(content, 'calls')
    df = yahoo_opt_clean(content, 'puts')
    options = options.append(df, ignore_index=True)
    for i in range(1, 18):
        content = requests.get(url + '?date=' + str(dates[i])).text
        content = json.loads(content)
        df = yahoo_opt_clean(content, 'calls')
        options = options.append(df, ignore_index=True)
        df = yahoo_opt_clean(content, 'puts')
        options = options.append(df, ignore_index=True)
    return options, current_price, current_date

options, current_price, date = get_options()

In [ ]:
options.drop(options[options.inTheMoney == True].index, inplace=True)
options['price'] = (options['ask'] - options['bid'])/2 + options['bid']
options.drop(['ask', 'bid', 'inTheMoney'], axis=1, inplace=True)
options.reset_index(drop=True, inplace=True)

In [ ]:
df = options.copy(deep=True)
# Pivot
df = df.pivot(index='expiration', columns='strike', values='price')
# Only keep columns with no NaN's
df = df[df.columns[~df.isna().any()]]
# Convert the index to number of days from today
date = dt.datetime.fromtimestamp(int(date))
df.index = pd.to_datetime(df.index, unit='s')
df.index = (df.index - date).days
# Drop rows more than 100 days out
df = df[df.index < 100]

In [ ]:
df.head()

In [ ]:
# Get interest rate
r = 0.05

In [ ]:
def d1d2(S, K, r, sigma, T):
    # Takes T in years
    d1 = (np.log(S / K) + ((r + ((sigma**2)/2))*T)) / (sigma * np.sqrt(T))
    d2 = d1 - (sigma * np.sqrt(T))
    return d1, d2


def price_call(S, K, r, sigma, T):
    T /= 365 # Converts T from days to years
    d1, d2 = d1d2(S, K, r, sigma, T)
    c = (S * norm.cdf(d1)) - (K * np.exp(-1 * r * T) * norm.cdf(d2))
    return c

def price_put(S, K, r, sigma, T):
    T /= 365 # Converts T from days to years
    d1, d2 = d1d2(S, K, r, sigma, T)
    c = (K * np.exp(-1 * r * T) * norm.cdf(-d2)) - (S * norm.cdf(-d1))
    return c

def option_vega(S, K, r, sigma, T):
    T /= 365 # Converts T from days to years
    d1, d2 = d1d2(S, K, r, sigma, T)
    v = S*np.sqrt(T)*norm.pdf(d1)/100 # IDK why I have to divide here
    return v

In [ ]:
def bs_estimate(iv, frame):
    df = frame.copy(deep=True)
    bs_price = np.zeros(df.shape)
    i = 0
    j = 0
    for rowval, row in df.iterrows():
        for colval, col in df.iteritems():
            if colval <= current_price:
                bs_price[i,j] = price_put(current_price, colval, r, iv[i, j], rowval)
            else:
                bs_price[i,j] = price_call(current_price, colval, r, iv[i, j], rowval)
            j += 1
        j = 0
        i += 1
    return bs_price

def vega_estimate(iv, frame):
    df = frame.copy(deep=True)
    vega = np.zeros(df.shape)
    i = 0
    j = 0
    for rowval, row in df.iterrows():
        for colval, col in df.iteritems():
            vega[i,j] = option_vega(current_price, colval, r, iv[i, j], rowval)
            j += 1
        j = 0
        i += 1
    return vega

iv = np.full(df.shape, 0.2)
bs_price = bs_estimate(iv, df)
res = bs_price - df.values
vega = vega_estimate(iv, df)

counter = 0
while(max(-1*res.min(), res.max()) > 0.00001):
    iv = iv - (res/vega)/500
    # Divide by 100 as a cheap hack to stop Newton's method from exploding
    # Should probably switch to bisection or secant or similar
    bs_price = bs_estimate(iv, df)
    res = bs_price - df.values
    vega = vega_estimate(iv, df)
    counter += 1
    print("Finished %d iterations" % counter)

In [ ]:
X = list(df)
Y = df.index.values
X, Y = np.meshgrid(X, Y)
fig = plt.figure(figsize=(10,8))
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, iv)
ax.xaxis.set_label_text('Strike')
ax.yaxis.set_label_text('Days until expiry')
title_text = "Implied volatility on %s" % dt.datetime.strftime(date, "%Y-%m-%d")
ax.set_title(title_text)
plt.show()